<h1>Prophet</h1>

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from fbprophet import Prophet
from sklearn.metrics import mean_squared_error
from numba import jit
import math

In [3]:
train_1 = pd.read_csv('../Data/train_1.csv')
train_1.head()

,Page,2015-07-01,2015-07-02,2015-07-03,2015-07-04,2015-07-05,2015-07-06,2015-07-07,2015-07-08,2015-07-09,...,2016-12-22,2016-12-23,2016-12-24,2016-12-25,2016-12-26,2016-12-27,2016-12-28,2016-12-29,2016-12-30,2016-12-31
0,2NE1_zh.wikipedia.org_all-access_spider,18.0,11.0,5.0,13.0,14.0,9.0,9.0,22.0,26.0,...,32.0,63.0,15.0,26.0,14.0,20.0,22.0,19.0,18.0,20.0
1,2PM_zh.wikipedia.org_all-access_spider,11.0,14.0,15.0,18.0,11.0,13.0,22.0,11.0,10.0,...,17.0,42.0,28.0,15.0,9.0,30.0,52.0,45.0,26.0,20.0
2,3C_zh.wikipedia.org_all-access_spider,1.0,0.0,1.0,1.0,0.0,4.0,0.0,3.0,4.0,...,3.0,1.0,1.0,7.0,4.0,4.0,6.0,3.0,4.0,17.0
3,4minute_zh.wikipedia.org_all-access_spider,35.0,13.0,10.0,94.0,4.0,26.0,14.0,9.0,11.0,...,32.0,10.0,26.0,27.0,16.0,11.0,17.0,19.0,10.0,11.0
4,52_Hz_I_Love_You_zh.wikipedia.org_all-access_s...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,48.0,9.0,25.0,13.0,3.0,11.0,27.0,13.0,36.0,10.0


In [4]:
train_1_reduce = train_1[0:100]
train_1_reduce = train_1_reduce.fillna(0)
train_1_reduce = train_1_reduce.set_index('Page')
train_1_reduce = train_1_reduce.T.rename_axis('Dates')
train_1_reduce.head()

Page,2NE1_zh.wikipedia.org_all-access_spider,2PM_zh.wikipedia.org_all-access_spider,3C_zh.wikipedia.org_all-access_spider,4minute_zh.wikipedia.org_all-access_spider,52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,5566_zh.wikipedia.org_all-access_spider,91Days_zh.wikipedia.org_all-access_spider,A'N'D_zh.wikipedia.org_all-access_spider,AKB48_zh.wikipedia.org_all-access_spider,ASCII_zh.wikipedia.org_all-access_spider,...,Lovelyz_zh.wikipedia.org_all-access_spider,Lulu_zh.wikipedia.org_all-access_spider,MADTOWN_zh.wikipedia.org_all-access_spider,MAMAMOO_zh.wikipedia.org_all-access_spider,MONSTA_X_zh.wikipedia.org_all-access_spider,Madame_Antoine_zh.wikipedia.org_all-access_spider,Mamamoo_zh.wikipedia.org_all-access_spider,Mike_D._Angelo_zh.wikipedia.org_all-access_spider,Miss_A_zh.wikipedia.org_all-access_spider,Missing9_zh.wikipedia.org_all-access_spider
Dates,,,,,,,,,,,,,,,,,,,,,
2015-07-01,18.0,11.0,1.0,35.0,0.0,12.0,0.0,118.0,5.0,6.0,...,2.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,3.0,0.0
2015-07-02,11.0,14.0,0.0,13.0,0.0,7.0,0.0,26.0,23.0,3.0,...,6.0,0.0,0.0,7.0,0.0,0.0,0.0,0.0,9.0,0.0
2015-07-03,5.0,15.0,1.0,10.0,0.0,4.0,0.0,30.0,14.0,5.0,...,5.0,0.0,1.0,17.0,0.0,0.0,0.0,0.0,12.0,0.0
2015-07-04,13.0,18.0,1.0,94.0,0.0,5.0,0.0,24.0,12.0,12.0,...,21.0,0.0,0.0,11.0,0.0,0.0,0.0,0.0,14.0,0.0
2015-07-05,14.0,11.0,0.0,4.0,0.0,20.0,0.0,29.0,9.0,6.0,...,4.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0,6.0,0.0


In [5]:
def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true[i]
        b = y_pred[i]
        c = a+b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [6]:
metrics_df = pd.DataFrame(columns=['RMSE','SMAPE','MAPE'])

for column in train_1_reduce.columns:
    print(column)
    df = pd.DataFrame(train_1_reduce[column])
    df = df.reset_index()
    df.columns = ['ds', 'y']
    dftrain = df[0:500]
    dftest = df[500:550]
    m = Prophet()
    m.fit(dftrain)
    future = m.make_future_dataframe(periods=50)
    future.tail()
    forecast = m.predict(future)
    forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]
    predictions = forecast['yhat'][500:550]
    rmse = np.sqrt(mean_squared_error(dftest['y'], predictions))
    smape = smape_fast(dftest['y'].values, predictions.values)
    mape = np.mean(np.abs(predictions.values - dftest['y'].values)/np.abs(dftest['y'].values))
    print("Test RMSE: %.3f" % rmse)
    print("Test SMAPE: %.3f" % smape)
    print("Test MAPE: %.3f" % mape)
    metrics_df.loc[column] = [rmse,smape,mape]

2NE1_zh.wikipedia.org_all-access_spider


INFO:numexpr.utils:NumExpr defaulting to 8 threads.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.
INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 29.651
Test SMAPE: 44.070
Test MAPE: 0.504
2PM_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 85.190
Test SMAPE: 65.973
Test MAPE: 1.054
3C_zh.wikipedia.org_all-access_spider


...

Test RMSE: 56.936
Test SMAPE: 56.844
Test MAPE: 0.661
Mike_D._Angelo_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 8.831
Test SMAPE: 50.495
Test MAPE: 0.566
Miss_A_zh.wikipedia.org_all-access_spider


INFO:fbprophet:Disabling yearly seasonality. Run prophet with yearly_seasonality=True to override this.
INFO:fbprophet:Disabling daily seasonality. Run prophet with daily_seasonality=True to override this.


Test RMSE: 5.804
Test SMAPE: 28.881
Test MAPE: 0.349
Missing9_zh.wikipedia.org_all-access_spider
Test RMSE: 21.376
Test SMAPE: 76.662
Test MAPE: 1.312


In [7]:
metrics_df

,RMSE,SMAPE,MAPE
2NE1_zh.wikipedia.org_all-access_spider,29.650920,44.069894,0.504006
2PM_zh.wikipedia.org_all-access_spider,85.189853,65.973280,1.054314
3C_zh.wikipedia.org_all-access_spider,29.410125,55.640537,inf
4minute_zh.wikipedia.org_all-access_spider,8.782234,42.136536,0.552176
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider,26.066972,46.243170,0.528003
...,...,...,...
Madame_Antoine_zh.wikipedia.org_all-access_spider,5.137325,126.467054,0.751732
Mamamoo_zh.wikipedia.org_all-access_spider,56.936363,56.843627,0.660532
Mike_D._Angelo_zh.wikipedia.org_all-access_spider,8.831422,50.494681,0.565722
Miss_A_zh.wikipedia.org_all-access_spider,5.803914,28.880674,0.349314


In [8]:
moyenne_RMSE = metrics_df['RMSE'].mean()
somme_RMSE = metrics_df['RMSE'].sum()
moyenne_SMAPE = metrics_df['SMAPE'].mean()
moyenne_MAPE = metrics_df['MAPE'].mean()

In [10]:
metrics_df['MAPE']

2NE1_zh.wikipedia.org_all-access_spider                0.504006
2PM_zh.wikipedia.org_all-access_spider                 1.054314
3C_zh.wikipedia.org_all-access_spider                       inf
4minute_zh.wikipedia.org_all-access_spider             0.552176
52_Hz_I_Love_You_zh.wikipedia.org_all-access_spider    0.528003
                                                         ...   
Madame_Antoine_zh.wikipedia.org_all-access_spider      0.751732
Mamamoo_zh.wikipedia.org_all-access_spider             0.660532
Mike_D._Angelo_zh.wikipedia.org_all-access_spider      0.565722
Miss_A_zh.wikipedia.org_all-access_spider              0.349314
Missing9_zh.wikipedia.org_all-access_spider            1.311642
Name: MAPE, Length: 100, dtype: float64

In [9]:
print("Test moyenne RMSE: %.3f" % moyenne_RMSE)
print("Test somme RMSE: %.3f" % somme_RMSE)
print("Test moyenne SMAPE: %.3f" % moyenne_SMAPE)
print("Test moyenne MAPE: %.3f" % moyenne_MAPE)

Test moyenne RMSE: 20.977
Test somme RMSE: 2097.731
Test moyenne SMAPE: 36.670
Test moyenne MAPE: inf
